In [ ]:
using ExprRules
using Plots
pgfplots()
include("inverse_logic.jl")

Define the LTL Grammar with "globally" and "eventually" operators

In [ ]:
grammar = @grammar begin
    R = (R && R) | (R || R) # "and" and "or" expressions for scalar values
    R = all(τ) | any(τ)# τ is true everywhere or τ is eventually true
    τ = (τ .& τ) | (τ .| τ) # "and" and "or" for boolean time series
    τ = (sym < c) | (sym > c) | (sym == c)# Less than operation
    c = 0.01 | 0.25 | 0.5 | 0.75 | 0.99 # Constants for comparison
    sym = x | y
end

Define two different action spaces. One for 1D and and one for 2D.

In [ ]:
A1D = ActionSpace(:x => [-1,1])
A2D = ActionSpace(:x => [0,1], :y => [0,1]);

Set the number of timesteps in the trajectory and then sample some unconstrained trajectories from the action spaces

In [ ]:
N = 100
a1_series = sample_series([A1D for i in 1:N])
a2_series = sample_series([A2D for i in 1:N]);

Example with 1D action space and simple constraints on values

In [ ]:
rn1 = RuleNode(1, [RuleNode(3,[RuleNode(8, [RuleNode(15), RuleNode(12)])]), RuleNode(4, [RuleNode(8, [RuleNode(15), RuleNode(14)])])])
ex1 = get_executable(rn1, grammar)
constrained_series = sample_series(ex1, A1D, N)
plot(a1_series, ylims = (-1,1), label="Unconstrained Series", size = (600,200), legend=:bottomright)
plot!(constrained_series, label = "Constrained Series")
title!(string("Time Series Constrained by: ", replace(string(ex1), "&" =>"\\&")))


Example with 2D action space

In [ ]:
r = RuleNode
rn2 = r(3, [r(6, [r(5,[r(8,[r(15), r(13)]), r(8,[r(16), r(13)])]),r(5,[r(7,[r(15), r(11)]), r(7,[r(16), r(11)])])])])
ex2 = get_executable(rn2, grammar)
constrained_series = sample_series(ex2, A2D, N)
p1 = plot(a2_series, ylims = (0,1), title="Unconstrained Series", label=["x", "y"])
p2 = plot(constrained_series, ylims = (0,1), label=["x", "y"], title = string("Series Constrained by: ", replace(string(ex2), "&" =>"\\&")))
plot(p1, p2, layout = (2,1))

A look inside:

In [ ]:
N = 4
leaves = eval_conditional_tree(ex1, true, N)

In [ ]:
constraints = gen_constraints(leaves, N)

In [ ]:
action_spaces, valid = gen_action_spaces(A1D, constraints)
println(action_spaces[1].bounds)
println("Is Valid? ", valid)

Next steps:
* Combine this with GP to solve a problem where the ground truth expression is known
* Apply it to the AV scenario
* Change the sampling approach to incorporate gaussian processes instead of just uniform, independent samples
* Generative modelfor the sign language dataset?

Hangups:
* Best way to sample continuous values and then incorporate them into GP?
* Expression generation can fail with the grammar because the algorithm doesn't keep track of how deep it is until right before it has reached the last depth. Talk to Ritchie?